In [1]:
import numpy as np
import pandas as pd

import datetime

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import re

from data_preprocess import DataTransformer
random_state = 12345

In [2]:


def features_interval(features, target, date1, date2):
    """
    Функция для выделения временных интервалов из таблиц признаков и целей
    на этом этапе отбрасываем колонку 'date'
    """
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    
    return features_interval, target_interval

In [3]:
def mae_day(y_true, y_pred):
    """
    Функция для вычисления метрики mae по дням из почасовых массивов данных
    """
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)

In [4]:
transformer = DataTransformer()

In [5]:
path = 'df_test_22222.csv'
all_ds, test_begin, test_end = transformer.open_file()

Файл не найден, попробуйте еще раз
начало открытого теста: 2023-04-01 00:00:00     конец открытого теста: 2023-08-01 00:00:00


In [6]:
all_ds = transformer.transform(all_ds)

В начале 2023-07-31
В date_transform 2023-07-31 00:00:00
В fill_weather_columns 2023-07-31 00:00:00
В holydays 2023-07-31 00:00:00
В create_lags 2023-07-31 00:00:00
В add_vvp2 2023-07-31 00:00:00
true wether before shift 2023-09-30 23:30:00
P
U
WW
Td
N
S
W
E
true wether after shift 2023-09-30 23:30:00
IS NAN 129
В add_true_weather 2023-07-31 00:00:00


In [ ]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 
feature_cols = list(all_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168'] 

for name in drop_list:
    feature_cols.remove(name)


In [ ]:
# Формируем набор датасетов для обучения и проверки

features = all_ds[feature_cols]
target = all_ds['target']

# готовим учебный и тестовый датасеты
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', test_begin)
features_test, target_test = features_interval(features, target, test_begin, test_end)

In [ ]:

# Здесь обучаем на всех данных, которые были нам предоставлены 
lgbm_model_all_train = lgb.LGBMRegressor(num_leaves=15, learning_rate=0.02, num_iterations=10000, 
                               feature_fraction=0.987, random_state=random_state, objective='regression_l1', n_jobs=-1)
lgbm_model_all_train.fit(features_all_train, target_all_train)


In [ ]:
predict = lgbm_model_all_train.predict(features_test)
predict_result = pd.DataFrame( features.loc[features_test.index, 'date'] )
predict_result['predict'] = predict
predict_result = predict_result.groupby(by='date').sum().reset_index()

In [ ]:
mae_day(target_test, predict)

In [ ]:
print(f'MAE = {mae_day}')